# 考虑用@property来代替属性重构

In [1]:
import logging
from datetime import datetime, timedelta

@property修饰器，使开发者可以把类设计得较为灵巧，从而令调用者能够轻松地访问该类的实例属性，此外，可以把简单的数值属性迁移为实时计算的属性。

**示例：**用纯Python对象实现带有配额的漏桶。

In [2]:
class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return 'Bucket(quota=%d)' % self.quota

In [3]:
bucket = Bucket(60)
print(bucket)

Bucket(quota=0)


漏桶算法需要满足：无论向桶中加多少水，都必须在进入下一个周期时将其清空。

In [4]:
def fill(bucket, amount):
    now = datetime.now()
    #如果到达下一个周期，清空桶里面的配额。
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

每次在执行消耗配额的操作之前，都必须先确认桶里有足够的配额可供使用。

In [5]:
def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

使用这个对象之前，要先往桶里添水。

In [6]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [7]:
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

Had 99 quota
Bucket(quota=1)


In [8]:
if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

Not enough for 3 quota
Bucket(quota=1)


**缺点：**以后无法得知漏桶里的初始配额。配额会在每个周期内持续流失，如果降到0，那么deduct就总会返回False，此时，依赖deduct的那些操作就会受到阻塞。

**解决办法：**在类中使用max_quota来记录本周期的初始配额，并且用quota_consumed来记录本周期内所消耗的配额。

In [9]:
class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0

    def __repr__(self):
        return ('Bucket(max_quota=%d, quota_consumed=%d)' %
                (self.max_quota, self.quota_consumed))
    
    # 实时算出当前所剩的配额
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    # 保证quota能够与该类接口中的fill和deduct相匹配
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [10]:
bucket = Bucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')

print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')

print('Still', bucket)

Initial Bucket(max_quota=0, quota_consumed=0)
Filled Bucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now Bucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still Bucket(max_quota=100, quota_consumed=99)
